## build factor matrixs

In [4]:
import os
import pandas as pd
from tqdm import tqdm

from joblib import Parallel, delayed

In [2]:
df = pd.read_csv('/data/QAE/GKX_20201231.csv')

In [5]:
factor_list = ['absacc','acc','aeavol' ,'age' ,'agr' ,'baspread' ,'beta' ,'betasq',
               'bm','bm_ia' ,'cash','cashdebt','cashpr','cfp','cfp_ia' ,'chatoia','chcsho' ,'chempia',
               'chinv','chmom' ,'chpmia' ,'chtx','cinvest','convind' ,'currat' ,'depr' ,'divi' ,
               'divo','dolvol' ,'dy','ear','egr','ep','gma','grcapx','grltnoa','herf','hire',
               'idiovol','ill','indmom','invest','lev','lgr','maxret','mom12m','mom1m','mom36m',
               'mom6m','ms','mvel1','mve_ia','nincr','operprof','orgcap','pchcapx_ia','pchcurrat',
               'pchdepr','pchgm_pchsale','pchquick','pchsale_pchinvt','pchsale_pchrect', 'pchsale_pchxsga', 
               'pchsaleinv', 'pctacc', 'pricedelay', 'ps', 'quick', 'rd', 'rd_mve', 'rd_sale', 
               'realestate', 'retvol', 'roaq', 'roavol', 'roeq', 'roic', 'rsup', 'salecash', 
               'saleinv', 'salerec', 'secured', 'securedind', 'sgr', 'sin', 'sp', 'std_dolvol', 
               'std_turn', 'stdacc', 'stdcf', 'tang', 'tb', 'turn', 'zerotrade', 'RET']

In [6]:
save_path = '/data/QAE/xiu_factor_matrix_with_na'

In [7]:
if not os.path.exists(save_path):
    os.mkdir(save_path)

In [9]:
def re_matrix(factor, df, save_path):
    permno_list = df['permno'].unique()
    permno_list.sort()
    date_list = df['DATE'].unique()
    date_list = date_list[(date_list >= 19570000) * (date_list <= 20161231)]
    date_list.sort()
    
    df_matrix = pd.DataFrame()
    df_matrix['date'] = date_list
    df_matrix.set_index('date', inplace = True)
    for permno in permno_list:
        df_ = df.loc[df['permno'] == permno]
        df_.set_index('DATE', inplace = True)
        df_.sort_index(inplace = True)
        df_matrix[permno] = df_[factor]
    # return df_matrix
    df_matrix.to_csv(f'{save_path}/{factor}.csv')

In [7]:
result_stock = Parallel(n_jobs = 10)(delayed(re_matrix)
                                     (factor, df, save_path) for factor in tqdm(factor_list))

100%|██████████| 94/94 [22:30<00:00, 14.37s/it]


## fill the missing values in factor matrixs

In [11]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

from joblib import Parallel, delayed

In [12]:
load_path = '/data/QAE/xiu_factor_matrix_with_na'
save_path = '/data/QAE/xiu_factor_matrix'
if not os.path.exists(save_path):
    os.mkdir(save_path)

In [13]:
factor_list = ['absacc','acc','aeavol' ,'age' ,'agr' ,'baspread' ,'beta' ,'betasq',
               'bm','bm_ia' ,'cash','cashdebt','cashpr','cfp','cfp_ia' ,'chatoia','chcsho' ,'chempia',
               'chinv','chmom' ,'chpmia' ,'chtx','cinvest','convind' ,'currat' ,'depr' ,'divi' ,
               'divo','dolvol' ,'dy','ear','egr','ep','gma','grcapx','grltnoa','herf','hire',
               'idiovol','ill','indmom','invest','lev','lgr','maxret','mom12m','mom1m','mom36m',
               'mom6m','ms','mvel1','mve_ia','nincr','operprof','orgcap','pchcapx_ia','pchcurrat',
               'pchdepr','pchgm_pchsale','pchquick','pchsale_pchinvt','pchsale_pchrect', 'pchsale_pchxsga', 
               'pchsaleinv', 'pctacc', 'pricedelay', 'ps', 'quick', 'rd', 'rd_mve', 'rd_sale', 
               'realestate', 'retvol', 'roaq', 'roavol', 'roeq', 'roic', 'rsup', 'salecash', 
               'saleinv', 'salerec', 'secured', 'securedind', 'sgr', 'sin', 'sp', 'std_dolvol', 
               'std_turn', 'stdacc', 'stdcf', 'tang', 'tb', 'turn', 'zerotrade', 'RET']

In [14]:
def fill_na(factor_name, load_path, save_path):
    df = pd.read_csv(f'{load_path}/{factor_name}.csv')
    df.set_index('date', inplace = True)
    df[(df == np.inf)] = np.nan
    df[(df == -np.inf)] = np.nan
    for j in range(df.shape[0]):
        a = df.iloc[j, :]
        a[a.isna()] = a.median()
        df.iloc[j, :] = a
    df.fillna(method = 'ffill', inplace = True, axis = 0)
    df.fillna(method = 'bfill', inplace = True, axis = 0)
    # return df
    df.to_csv(f'{save_path}/{factor_name}.csv')

In [5]:
c = Parallel(n_jobs = 10)(delayed(fill_na)(factor_name, load_path, save_path) for factor_name in tqdm(factor_list))

100%|██████████| 94/94 [02:31<00:00,  1.61s/it] 


In [7]:
def test_na(factor_name, load_path):
    df = pd.read_csv(f'{load_path}/{factor_name}.csv')
    return df.isna().sum().sum()

In [8]:
c = Parallel(n_jobs = 10)(delayed(test_na)(factor_name, save_path) for factor_name in tqdm(factor_list))

100%|██████████| 94/94 [01:07<00:00,  1.40it/s]


In [15]:
fill_na('RET', load_path, save_path)